In [13]:
import pandas as pd
import numpy as np
import os
import re

import json
import slack
from datetime import datetime
from simple_salesforce import Salesforce, SalesforceLogin, SFType

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [14]:
### Salesforce ###
logininfo = json.load(open('salesforce_login.json'))

username = logininfo['username']
password = logininfo['password']
security_token = logininfo['security_token']
domain = 'login'

sf = Salesforce(username=username, password=password, security_token=security_token)

### Slack ###
slack_token = json.load(open('/ghds/groups/labdesk/bshih/slack_login.json'))['SLACK_TOKEN']
client = slack.WebClient(token=slack_token)

# client.chat_postMessage(channel='#bot_testing', text='CAN I GET A OOOOO YEAAA')

In [15]:
def sample_finder(all_samples):
    samples = all_samples[all_samples['run_sample_id'].str.startswith(('G', 'H'), na=False)]
    samples = samples[~samples['run_sample_id'].str.startswith('Ht', na=False)]
    filtered = samples.groupby('run_sample_id').filter(lambda x: (x['rnase_count'].median() <= 50) & (x['covid_ratio'].median() >= 0.01))

    return filtered

def salesforce_query(samples):
    weird_samples = []
    for i in samples['run_sample_id'].unique():
        weird_samples.extend(sf.query_all(f"SELECT GH_Sample_ID__c, Status, Specimen_Collection_Date_Time__c, Specimen_receipt_date__c,\
                  State_Authorities_Notified_Date__c, Site_Name__c \
                  FROM Order WHERE GH_Sample_ID__c = '{i}'").get('records'))

    dataframe = pd.DataFrame(weird_samples)
    df = dataframe.drop(columns='attributes').rename(columns={'GH_Sample_ID__c':'GH Sample ID',
                                                       'Specimen_Collection_Date_Time__c':'Specimen Collection Date/Time',
                                                       'Specimen_receipt_date__c':'Specimen receipt date',
                                                       'State_Authorities_Notified_Date__c':'State Authorities Notified Date',
                                                       'Site_Name__c':'Site Name'})
    df['Specimen Collection Date/Time'] = pd.to_datetime(df['Specimen Collection Date/Time']).dt\
                                            .tz_convert('US/Pacific').dt.strftime('%-m/%-d/%Y, %-I:%M %p')
    df['State Authorities Notified Date'] = pd.to_datetime(df['State Authorities Notified Date']).dt\
                                            .tz_convert('US/Pacific').dt.strftime('%-m/%-d/%Y, %-I:%M %p')
    df['Specimen receipt date'] = pd.to_datetime(df['Specimen receipt date']).dt.strftime('%-m/%-d/%y')

    return df

def merge_sample_salesforce(samples, salesforce_data):
    df = pd.merge(samples, salesforce_data, left_on='run_sample_id', right_on='GH Sample ID', how='left').loc[:,['runid',
                                                                                                                'run_sample_id', 
                                                                                                                'covid_ratio', 
                                                                                                                'covid_count', 
                                                                                                                'rnase_count', 
                                                                                                                'spikein_count', 
                                                                                                                'replicate_call', 
                                                                                                                'replicate_flags', 
                                                                                                                'Specimen Collection Date/Time', 
                                                                                                                'Specimen receipt date', 
                                                                                                                'State Authorities Notified Date', 
                                                                                                                'Site Name']]\
                                                                                                            .sort_values(by=['runid', 'run_sample_id', 'covid_ratio'], ascending=[False, True, True])
    
    median_scores = df.groupby('run_sample_id').median().rename(columns={'covid_ratio':'new_median'})[['new_median']].round(2)
    df = pd.merge(df, median_scores, left_on='run_sample_id', right_on='run_sample_id', how='left')
    median = df.pop('new_median')
    df.insert(2, 'new_median', median)
    
    df['group'] = (df['run_sample_id'].shift() != df['run_sample_id']).cumsum()
    df.loc[df.duplicated('group'), ['runid', 'run_sample_id', 'new_median', 'Specimen Collection Date/Time', 'Specimen receipt date', 'State Authorities Notified Date', 'Site Name', 'new_median']] = ''
    df.drop(columns=['group'], inplace=True)
    
    df = df.astype({'covid_count':'int', 'rnase_count':'int', 'spikein_count':'int'}).round({'covid_ratio':2})
    return df
    
def create_pdf(samples, file_name):
    fig, ax =plt.subplots(figsize=(35,len(samples)/2))
    ax.axis('tight')
    ax.axis('off')
    the_table = ax.table(cellText=samples.values,colLabels=samples.columns,loc='center', rowLoc='right',bbox=[0, 0, 1, 1])

    the_table.auto_set_font_size(False)
    the_table.set_fontsize(12)

    for i in range(0, len(blah.columns)):
        [the_table[(j+k, i)].set_facecolor("#e0e0e0") for j in range(4, len(blah), 6) for k in range(3)]

    the_table.auto_set_column_width(col=list(range(len(samples.columns))))

    pp = PdfPages(f"/ghds/groups/labdesk/bshih/{file_name}.pdf")
    pp.savefig(fig, bbox_inches='tight', dpi=300)
    pp.close()
    
def slack_upload(fcid, file_name):
    filepath = f'/ghds/groups/labdesk/bshih/{file_name}.pdf'

    try:
        result = client.files_upload(
            channels='#g19_sample_observation',
            file=filepath,
            initial_comment=f"{fcid}: These samples have RNAse Count <= 50 and G19 Score >= 0.01")
        assert result["file"]
        print('File Successfully Uploaded to Slack!')
    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        assert e.response["ok"] is False
        assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
        print(f"Got an error: {e.response['error']}")
    

In [16]:
all_samples_singlicate = pd.read_pickle('/ghds/groups/labdesk/bshih/c19dash/c19_dashboard/c19_read_counts.pickle')
all_samples = sample_finder(all_samples_singlicate)
salesforce_data = salesforce_query(all_samples)
blah = merge_sample_salesforce(all_samples, salesforce_data)
# create_pdf(blah, 'samples_test')
# slack_upload('fcid', 'samples_test')

In [17]:
all_samples

,runid,run_sample_id,sample_type,parameter_set,index,barcode,covid_count,rnase_count,spikein_count,unknown,...,flags,pos_lamp,lamp_barcode,read_count,covid_n2_count,covid_s2_count,exact_count,exact_fraction,proximity,Unnamed: 0
575,201215_NB552398_0054_AHHJMKBGXG,HCV0667,Sample,Guardant19-RTPCR,CAATCTGG_TTGTTCCA,GGTTCAATGA_GAAGCGCCTT,61116.0,24.0,4.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,201215_NB552398_0054_AHHJMKBGXG,HCV0667,Sample,Guardant19-RTPCR,GATCCTGC_TTCCCAGC,TCGACAGTTA_AGGTACCATC,83598.0,10.0,5.0,144.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,201215_NB552398_0054_AHHJMKBGXG,HCV0667,Sample,Guardant19-RTPCR,CGAGGAAC_CCACTGGA,TCCACCGAGA_AACTGAACTG,54143.0,799.0,9.0,146.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1380,201215_NB552398_0054_AHHJMKBGXG,HBU0065,Sample,Guardant19-RTPCR,CGTCTAAG_GTCTTCAT,CTAGCTACTT_GCGTTAATGC,24063.0,0.0,156.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1528,201215_NB552398_0054_AHHJMKBGXG,HBU0065,Sample,Guardant19-RTPCR,GCTGGGCC_GAGAGATG,AACGGAGAGG_GCTTCTAGTC,32901.0,0.0,97.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1680,201215_NB552398_0054_AHHJMKBGXG,HBU0065,Sample,Guardant19-RTPCR,CCTACTTA_GTATGGAG,GGTACTTCAG_GTAGTAGTAC,42783.0,0.0,58.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3588,200919_NB551650_0127_AHH5KHBGXG,G000UFY,Sample,Guardant19-RTPCR,CAATCTGG_TTGTTCCA,GGTCCTATAT_GACTACCGTC,6.0,0.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,200919_NB551650_0127_AHH5KHBGXG,G000UFY,Sample,Guardant19-RTPCR,GATCCTGC_TTCCCAGC,ACTCTGTTGA_GTGATCTTAC,13.0,1.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3905,200919_NB551650_0127_AHH5KHBGXG,G000UFY,Sample,Guardant19-RTPCR,CGAGGAAC_CCACTGGA,AATGGTCTGA_CACGGTATCA,1.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5346,201205_NB552393_0045_AHHFTKBGXG,HCB0506,Sample,Guardant19-RTPCR,CGTCTAAG_GTCTTCAT,CGAAGATGAA_CCGATCAGTA,7176.0,9.0,28023.0,14668.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
all_samples_singlicate.head(50)

,runid,run_sample_id,sample_type,parameter_set,index,barcode,covid_count,rnase_count,spikein_count,unknown,...,flags,pos_lamp,lamp_barcode,read_count,covid_n2_count,covid_s2_count,exact_count,exact_fraction,proximity,Unnamed: 0
0,201212_NB552398_0053_AHHGFGBGXG,HBU0161,Sample,Guardant19-RTPCR,CTATTTGC_ACACTGAG,ACGACCGATA_TGTTCGTGGC,0.0,8345.0,33723.0,3213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201212_NB552398_0053_AHHGFGBGXG,HBU0410,Sample,Guardant19-RTPCR,CTATTTGC_ACACTGAG,CCATTACCAA_ACCTTCCTTG,0.0,7007.0,23897.0,807.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201212_NB552398_0053_AHHGFGBGXG,NaN,NaN,Guardant19-RTPCR,CTATTTGC_ACACTGAG,AATTGGATCC_CGGAACACGA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201212_NB552398_0053_AHHGFGBGXG,NaN,NaN,Guardant19-RTPCR,CTATTTGC_ACACTGAG,AGCTTACTGA_AGCTATGCCA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201212_NB552398_0053_AHHGFGBGXG,HBU0517,Sample,Guardant19-RTPCR,CTATTTGC_ACACTGAG,AGGCCATGAC_GCCTACAGGA,0.0,5623.0,43226.0,1609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201212_NB552398_0053_AHHGFGBGXG,HBV0045,Sample,Guardant19-RTPCR,CTATTTGC_ACACTGAG,TGCAAGTCCG_GCAAGACGGA,1.0,4467.0,15838.0,909.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201212_NB552398_0053_AHHGFGBGXG,NaN,NaN,Guardant19-RTPCR,CTATTTGC_ACACTGAG,ACAATGCTCA_CACGTAGACT,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,201212_NB552398_0053_AHHGFGBGXG,NaN,NaN,Guardant19-RTPCR,CTATTTGC_ACACTGAG,TCTTGCGTTA_CAGTAGGCGA,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,201212_NB552398_0053_AHHGFGBGXG,HCB0688,Sample,Guardant19-RTPCR,CTATTTGC_ACACTGAG,CAGGAGAGAA_CGGCATGATC,0.0,4067.0,35644.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,201212_NB552398_0053_AHHGFGBGXG,NaN,NaN,Guardant19-RTPCR,CTATTTGC_ACACTGAG,GAAGTGGCCT_ATGGCTGTGT,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
all_samples.to_csv('/ghds/groups/labdesk/bshih/clinopstest/210103_NB552478_0010_AHJ3KKBGXG_alltrue/c19_read_counts.csv')

In [24]:
all_samples_singlicate.head(50).to_csv('/ghds/groups/labdesk/bshih/clinopstest/210103_NB552478_0010_AHJ3KKBGXG_allfalse/c19_read_counts.csv')

In [ ]:
import json
import logging
import re
import time
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import watchdog.events
from matplotlib.backends.backend_pdf import PdfPages
from simple_salesforce import Salesforce
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from watchdog.observers.polling import PollingObserver as Observer

### Salesforce API Login ###
logininfo = json.load(open('/ghds/groups/labdesk/bshih/salesforce_login.json'))

username = logininfo['username']
password = logininfo['password']
security_token = logininfo['security_token']
domain = 'login'

sf = Salesforce(username=username, password=password, security_token=security_token)

### Slack Token ###
slack_token = json.load(open('/ghds/groups/labdesk/bshih/slack_login.json'))['SLACK_TOKEN']
client = WebClient(token=slack_token)

### Logger Setup ###
# LOGGER_NAME = 'clinical_observation2'
# LOG_FILE_DIR = '/ghds/groups/labdesk/bshih'
LOGGER_NAME = 'clinical_observation_demonstration'
LOG_FILE_DIR = '/home/bshih/'

logger = logging.getLogger(LOGGER_NAME)

# log_file = f"{LOG_FILE_DIR}/clinical_observation2.log"
log_file = f"{LOG_FILE_DIR}/clinical_observation_demonstration.log"
handler = logging.FileHandler(log_file)

handler.setLevel(logging.INFO)
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)


class Handler(watchdog.events.FileSystemEventHandler):
    def __init__(self):
        watchdog.events.FileSystemEventHandler()
        logger.info(f'Program has been initiated!')

    @staticmethod
    def sample_finder(all_samples):
        """
        :param all_samples:
        :return: Filtered dataframe with samples whose RNAse <= 50 and G19 >= 0.01, singlicates.
        """
        samples = all_samples[all_samples['run_sample_id'].str.startswith(('G', 'H'), na=False)]
        samples = samples[~samples['run_sample_id'].str.startswith('Ht', na=False)]
        filtered = samples.groupby('run_sample_id').filter(
            lambda x: (x['rnase_count'].median() <= 50) & (x['covid_ratio'].median() >= 0.01))

        return filtered

    @staticmethod
    def salesforce_query(samples):
        """
        :param samples:
        :return: Salesforce query of filtered samples
        """
        weird_samples = []
        for i in samples['run_sample_id'].unique():
            weird_samples.extend(sf.query_all(f"SELECT GH_Sample_ID__c, Status, Specimen_Collection_Date_Time__c, Specimen_receipt_date__c,\
                      State_Authorities_Notified_Date__c, Site_Name__c \
                      FROM Order WHERE GH_Sample_ID__c = '{i}'").get('records'))

        dataframe = pd.DataFrame(weird_samples)
        df = dataframe.drop(columns='attributes').rename(columns={'GH_Sample_ID__c': 'GH Sample ID',
                                                                  'Specimen_Collection_Date_Time__c': 'Specimen Collection Date/Time',
                                                                  'Specimen_receipt_date__c': 'Specimen receipt date',
                                                                  'State_Authorities_Notified_Date__c': 'State Authorities Notified Date',
                                                                  'Site_Name__c': 'Site Name'})
        df['Specimen Collection Date/Time'] = pd.to_datetime(df['Specimen Collection Date/Time']).dt \
            .tz_convert('US/Pacific').dt.strftime('%-m/%-d/%Y, %-I:%M %p')
        df['State Authorities Notified Date'] = pd.to_datetime(df['State Authorities Notified Date']).dt \
            .tz_convert('US/Pacific').dt.strftime('%-m/%-d/%Y, %-I:%M %p')
        df['Specimen receipt date'] = pd.to_datetime(df['Specimen receipt date']).dt.strftime('%-m/%-d/%y')

        return df

    @staticmethod
    def merge_sample_salesforce(samples, salesforce_data):
        """
        :param samples:
        :param salesforce_data:
        :return: Merged dataframe samples with salesforce data
        """
        df = pd.merge(samples, salesforce_data, left_on='run_sample_id', right_on='GH Sample ID', how='left').loc[:,
             ['runid',
              'run_sample_id',
              'covid_ratio',
              'covid_count',
              'rnase_count',
              'spikein_count',
              'replicate_call',
              'replicate_flags',
              'Specimen Collection Date/Time',
              'Specimen receipt date',
              'State Authorities Notified Date',
              'Site Name']] \
            .sort_values(by=['runid', 'run_sample_id', 'covid_ratio'], ascending=[False, True, True])

        median_scores = df.groupby('run_sample_id').median().rename(columns={'covid_ratio': 'new_median'})[
            ['new_median']].round(2)
        df = pd.merge(df, median_scores, left_on='run_sample_id', right_on='run_sample_id', how='left')
        median = df.pop('new_median')
        df.insert(2, 'new_median', median)

        df['group'] = (df['run_sample_id'].shift() != df['run_sample_id']).cumsum()
        df.loc[df.duplicated('group'), ['runid', 'run_sample_id', 'new_median', 'Specimen Collection Date/Time',
                                        'Specimen receipt date', 'State Authorities Notified Date', 'Site Name',
                                        'new_median']] = ''
        df.drop(columns=['group'], inplace=True)

        df = df.astype({'covid_count': 'int', 'rnase_count': 'int', 'spikein_count': 'int'}).round({'covid_ratio': 2})
        return df

    @staticmethod
    def create_pdf(samples, file_name):
        fig, ax = plt.subplots(figsize=(35, len(samples) // 2))
        ax.axis('tight')
        ax.axis('off')
        the_table = ax.table(cellText=samples.values, colLabels=samples.columns, loc='center', rowLoc='right',
                             bbox=[0, 0, 1, 1])

        the_table.auto_set_font_size(False)
        the_table.set_fontsize(12)

        for i in range(0, len(samples.columns)):
            [the_table[(j + k, i)].set_facecolor("#e0e0e0") for j in range(4, len(samples), 6) for k in range(3)]

        the_table.auto_set_column_width(col=list(range(len(samples.columns))))

        pp = PdfPages(f"/ghds/groups/labdesk/bshih/{file_name}.pdf")
        pp.savefig(fig, bbox_inches='tight', dpi=300)
        pp.close()

    @staticmethod
    def slack_upload(fcid, file_name):
        filepath = f'/ghds/groups/labdesk/bshih/{file_name}.pdf'

        try:
            result = client.files_upload(
                channels='#g19_sample_observation',
                file=filepath,
                initial_comment=f"{fcid}: These samples have RNAse Count <= 50 and G19 Score >= 0.01")
            assert result["file"]
            logger.info('File Successfully Uploaded to Slack!')
        except SlackApiError as e:
            # You will get a SlackApiError if "ok" is False
            assert e.response["ok"] is False
            assert e.response["error"]  # str like 'invalid_auth', 'channel_not_found'
            logger.info(f"Got an error: {e.response['error']}")

    def on_any_event(self, event):
        logger.info(f'This file has been changed. Source path --> {event.src_path}')

        today = datetime.strftime(datetime.today(), '%y%m%d')
#         if bool(re.match(f'^.*{today}.*c19_read_counts.hdr.tsv?', event.src_path)):
        if bool(re.match(f'^.*210103.*c19_read_counts.csv?', event.src_path)):

#             flowcell = event.src_path[20:51]
            flowcell = event.src_path[39:70]

            logger.info(f'Flowcell {flowcell} c19_read_counts created!')

#             current_read_counts = pd.read_csv(event.src_path, sep='\t')
            current_read_counts = pd.read_csv(event.src_path)
            low_rnase_high_covid = Handler.sample_finder(current_read_counts)

            if len(low_rnase_high_covid) != 0:
                logger.info(f'There are anomaly samples in: {flowcell}')

                salesforce_data = Handler.salesforce_query(low_rnase_high_covid)
                merged_data = Handler.merge_sample_salesforce(low_rnase_high_covid, salesforce_data)

                filename = flowcell + '_lowRNAse_COVIDpositive'

                Handler.create_pdf(merged_data, filename)
                Handler.slack_upload(flowcell, filename)

                logger.info(f'Salesforce API Usage: {sf.api_usage}')

            else:
                client.chat_postMessage(channel='#g19_sample_observation',
                                        text=f'{flowcell}: No QC fail samples that are covid positive today! Have a great day :)')
                logger.info('No QC fail samples that are covid positive today! Have a great day :)')


if __name__ == "__main__":
#     src_path = "/ghds/cv19/analysis"
    src_path = "/ghds/groups/labdesk/bshih/"
    event_handler = Handler()
    observer = Observer()
    observer.schedule(event_handler, path=src_path, recursive=True)
    observer.start()
    try:
        while True:
            logger.info("Program is running every 1 Hour!")
            time.sleep(3600)
    except KeyboardInterrupt:
        observer.stop()
        observer.join()


In [ ]:
path = '/ghds/cv19/analysis/210101_NB552478_0010_AHJ3KKBGXG.02ab0e87-1aa3-44ea-8ea6-a1f1e1eb7447.20210101144150/c19_read_counts.hdr.tsv'
today = datetime.strftime(datetime.today(), '%y%m%d')

if bool(re.match(f'^.*{today}.*c19_read_counts.hdr.tsv?', path)):
    print('hi')

# TEST LOGGING

In [ ]:
import logging

LOGGER_NAME = 'clinical_observation'
LOG_FILE_DIR = '/ghds/groups/labdesk/bshih'
    
logger = logging.getLogger(LOGGER_NAME)

log_file = f"{LOG_FILE_DIR}/clinical_observation.log"
handler = logging.FileHandler(log_file)

handler.setLevel(logging.INFO)
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", datefmt='%Y-%m-%d %H:%M:%S')
handler.setFormatter(formatter)
logger.addHandler(handler)  

In [ ]:
from datetime import datetime
def main():
    logger.info(f"Program is running!")

if __name__ == "__main__":
    main()